# Categorical Naive Bayes 2

- Same as Categorical Naive Bayes, but with different combinations of predictors

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas
import numpy
import matplotlib.pyplot as plt
import itertools
import sklearn
from sklearn.metrics import log_loss

In [7]:
import loan_approval_lib
from loan_approval_lib import logspace, linspace

In [8]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.naive_bayes import CategoricalNB

In [9]:
import itertools

In [10]:
data = loan_approval_lib.load_original_data()

In [11]:
data

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,0.59,Y,3,1
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0.10,N,2,0
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,0.57,N,3,1
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,0.53,N,2,1
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,0.55,Y,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0.11,N,30,0
32577,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0.15,N,19,0
32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,0.46,N,28,1
32579,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0.10,N,26,0


In [12]:
data_test = loan_approval_lib.load_data()['test']

In [13]:
data_test

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2
1,58646,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,Y,4
2,58647,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,Y,2
3,58648,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,N,7
4,58649,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4
...,...,...,...,...,...,...,...,...,...,...,...,...
39093,97738,22,31200,MORTGAGE,2.0,DEBTCONSOLIDATION,B,3000,10.37,0.10,N,4
39094,97739,22,48000,MORTGAGE,6.0,EDUCATION,A,7000,6.03,0.15,N,3
39095,97740,51,60000,MORTGAGE,0.0,PERSONAL,A,15000,7.51,0.25,N,25
39096,97741,22,36000,MORTGAGE,4.0,PERSONAL,D,14000,15.62,0.39,Y,4


# Categorical NB Classifier

In [40]:
categorical_predictors = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
categorical_predictor_combinations = []

for count in range(1, len(categorical_predictors) + 1):
    for combination in itertools.combinations(categorical_predictors, count):
        categorical_predictor_combinations.append(list(combination))

#for combination in categorical_predictor_combinations:
#    print(len(combination), '  ', combination)

data_categorical = [
    data[categorical_predictor_combination].copy() for categorical_predictor_combination in categorical_predictor_combinations
]

len(data_categorical)

15

In [41]:
for dataset in data_categorical:

    data_categorical_columns = dataset.columns

    encoder = OrdinalEncoder()
    encoder.fit(dataset.to_numpy())
    dataset_encoded = encoder.transform(dataset.to_numpy())
    X = dataset_encoded
    
    dataset['loan_status'] = data['loan_status']
    y = dataset['loan_status']

    clf = CategoricalNB()
    clf.fit(X, y)
    dataset['loan_status_predict'] = clf.predict(dataset_encoded)
    
    P = (dataset['loan_status'] == dataset['loan_status_predict']).sum() / len(dataset)

    print(f'{', '.join(data_categorical_columns)}: P = {P}')

person_home_ownership: P = 0.7818360394094718
loan_intent: P = 0.7818360394094718
loan_grade: P = 0.815444584266904
cb_person_default_on_file: P = 0.7818360394094718
person_home_ownership, loan_intent: P = 0.7818360394094718
person_home_ownership, loan_grade: P = 0.826555354347626
person_home_ownership, cb_person_default_on_file: P = 0.7755133359933704
loan_intent, loan_grade: P = 0.8189742487953101
loan_intent, cb_person_default_on_file: P = 0.7818360394094718
loan_grade, cb_person_default_on_file: P = 0.815444584266904
person_home_ownership, loan_intent, loan_grade: P = 0.8360701022068077
person_home_ownership, loan_intent, cb_person_default_on_file: P = 0.7860409441085295
person_home_ownership, loan_grade, cb_person_default_on_file: P = 0.82462171204076
loan_intent, loan_grade, cb_person_default_on_file: P = 0.8199871090512876
person_home_ownership, loan_intent, loan_grade, cb_person_default_on_file: P = 0.8114852214480832


In [106]:
# testing - home ownership
data_tmp = data[['person_home_ownership', 'loan_status']].copy()

for key, df in data_tmp.groupby(by=['person_home_ownership', 'loan_status']):
    print(key)
    print(df)
    break

data_tmp.groupby(['person_home_ownership', 'loan_status']).count().reset_index()

('MORTGAGE', np.int64(0))
      person_home_ownership  loan_status
15                 MORTGAGE            0
34                 MORTGAGE            0
42                 MORTGAGE            0
45                 MORTGAGE            0
46                 MORTGAGE            0
...                     ...          ...
32569              MORTGAGE            0
32573              MORTGAGE            0
32576              MORTGAGE            0
32577              MORTGAGE            0
32579              MORTGAGE            0

[11754 rows x 2 columns]


,person_home_ownership,loan_status
0,MORTGAGE,0
1,MORTGAGE,1
2,OTHER,0
3,OTHER,1
4,OWN,0
5,OWN,1
6,RENT,0
7,RENT,1


In [42]:
# testing

#categorical_predictors = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
categorical_predictors = ['person_home_ownership', 'loan_intent']

data_categorical = data[categorical_predictors].copy()

dataset = data_categorical

encoder = OrdinalEncoder()
encoder.fit(dataset.to_numpy())
dataset_encoded = encoder.transform(dataset.to_numpy())
X = dataset_encoded

data_categorical['loan_status'] = data['loan_status']
y = dataset['loan_status']

clf = CategoricalNB()
clf.fit(X, y)
dataset['loan_status_predict'] = clf.predict(dataset_encoded)
P = (dataset['loan_status'] == dataset['loan_status_predict']).sum() / len(dataset)

print(f'{', '.join(categorical_predictors)}: P = {P}')

person_home_ownership, loan_intent: P = 0.7818360394094718


In [44]:
# verification

categorical_predictors = ['person_home_ownership', 'loan_intent']

data_categorical = data[categorical_predictors].copy()
data_categorical['loan_status'] = data['loan_status']
data_categorical

,person_home_ownership,loan_intent,loan_status
0,RENT,PERSONAL,1
1,OWN,EDUCATION,0
2,MORTGAGE,MEDICAL,1
3,RENT,MEDICAL,1
4,RENT,MEDICAL,1
...,...,...,...
32576,MORTGAGE,PERSONAL,0
32577,MORTGAGE,PERSONAL,0
32578,RENT,HOMEIMPROVEMENT,1
32579,MORTGAGE,PERSONAL,0


In [57]:
data_categorical_approve = data_categorical[data_categorical['loan_status'] == 0]
data_categorical_reject = data_categorical[data_categorical['loan_status'] == 1]

In [58]:
', '.join(sorted(data_categorical['person_home_ownership'].unique().tolist()))

'MORTGAGE, OTHER, OWN, RENT'

In [59]:
', '.join(sorted(data_categorical['loan_intent'].unique().tolist()))

'DEBTCONSOLIDATION, EDUCATION, HOMEIMPROVEMENT, MEDICAL, PERSONAL, VENTURE'

In [63]:
count1 = len(
    data_categorical_approve[
        (data_categorical_approve['person_home_ownership'] == 'MORTGAGE') &
        (data_categorical_approve['loan_intent'] == 'DEBTCONSOLIDATION')
    ]
)
count1

1864

In [64]:
count2 = len(
    data_categorical_reject[
        (data_categorical_reject['person_home_ownership'] == 'MORTGAGE') &
        (data_categorical_reject['loan_intent'] == 'DEBTCONSOLIDATION')
    ]
)
count2

448

In [76]:
P = (count2) / (count1 + count2)
P

0.19377162629757785

In [83]:
count_mortgage = len(
    data_categorical[
        data_categorical['person_home_ownership'] == 'MORTGAGE'
    ]
)
count_mortgage

13444

In [84]:
count_debtconsolidation = len(
    data_categorical[
        data_categorical['loan_intent'] == 'DEBTCONSOLIDATION'
    ]
)
count_debtconsolidation

5212

In [88]:
count_data = len(data_categorical)
count_data

32581

In [86]:
P_mortgage = count_mortgage / count_data
P_mortgage

0.412633129738191

In [87]:
P_debtconsolidation = count_debtconsolidation / count_data
P_debtconsolidation

0.15997053497437155

In [ ]:
count_mortgage_given_accept = len(data_categorical[data_categorical['loan_status']])
count_mortgage_given_accept

In [ ]:
count_debtconsolidation_given_accept = 
count_debtconsolidation_given_accept

In [90]:
count_accept = len(data_categorical[data_categorical['loan_status'] == 0])
count_accept

25473

In [ ]:
P_mortgage_given_accept = count_mortgage_given_accept / count_accept
P_mortgage_given_accept

In [ ]:
P_debtconsolidation_given_accept = count_debtconsolidation_given_accept / count_accept
P_debtconsolidation_given_accept

In [ ]:
P_mortgage_and_debtconsolidation_given_accept = P_mortgage_given_accept * P_debtconsolidation_given_accept
P_mortgage_and_debtconsolidation_given_accept

In [ ]:
P_accept_given_mortgage_and_debtconsolidation = P_mortgage_and_debtconsolidation_given_accept / (P_mortgage_and_debtconsolidation * P_accept)
P_accept_given_mortgage_and_debtconsolidation

In [73]:
# testing how it works
data_mytest = pandas.DataFrame(
    {
        'person_home_ownership': ['MORTGAGE'],
        'loan_intent': ['DEBTCONSOLIDATION'],
    }
)

data_categorical_ = data[['person_home_ownership', 'loan_intent']].copy()

dataset_ = data_categorical_
columns_ = dataset_.columns

encoder = OrdinalEncoder()
encoder.fit(dataset_.to_numpy())
dataset_encoded_ = encoder.transform(dataset_.to_numpy())
X = dataset_encoded_

dataset_['loan_status'] = data['loan_status']
y = dataset_['loan_status']

clf = CategoricalNB()
clf.fit(X, y)
dataset_['loan_status_predict'] = clf.predict(dataset_encoded_)

P = (dataset_['loan_status'] == dataset_['loan_status_predict']).sum() / len(dataset_)

print(f'{', '.join(columns_)}: P = {P}')

data_mytest_encoded = encoder.transform(data_mytest.to_numpy())
print(data_mytest)
print(data_mytest_encoded)
print(clf.predict(data_mytest_encoded))
print(clf.predict_proba(data_mytest_encoded))

person_home_ownership, loan_intent: P = 0.7818360394094718
[['MORTGAGE' 'DEBTCONSOLIDATION']]
[[0. 0.]]
[0]
[[0.82901424 0.17098576]]
